In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
import tensorflow as tf

import json
from pathlib import Path

/usr/lib64/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2023-11-22 17:18:55.709944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 17:18:55.710007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 17:18:55.943053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 17:18:56.460825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This 

In [2]:
path_to_training = Path("training")
path_to_test = Path("test")

## Données X et y

In [3]:
X = [] # contanation de toutes les discussions 
dialogues = path_to_training.glob('*.json')
dialogue_ids = []
for dialogue in dialogues:
    dialogue_ids.append(dialogue.name[:-5])
    with open(dialogue, 'r') as f:
        dialogue_json = json.load(f)
        X.extend([replique['speaker'] + ": " + replique['text'] for replique in dialogue_json])

In [4]:
y = [] # concatenation des labels
with open("training_labels.json", 'r') as f:
    labels = json.load(f)

for dialogue_id in dialogue_ids:
    y.extend(labels[dialogue_id])

In [5]:
len(X)==len(y)

True

## X_train, X_valid

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
#max_features = max([len(txt.split()) for txt in X_train])
max_features = 1700

vectorizer = TfidfVectorizer(max_features=max_features)
X_train_trans = vectorizer.fit_transform(X_train)
X_valid_trans = vectorizer.transform(X_valid)

X_train_trans = X_train_trans.toarray()
X_valid_trans = X_valid_trans.toarray()

y_train = np.array(y_train)

## Premier modele avec RandomForest

In [8]:
clf = RandomForestClassifier()
clf.fit(X_train_trans, y_train)

y_pred = clf.predict(X_valid_trans)
print("le f1_score est: "+ str(f1_score(y_valid, y_pred)))

le f1_score est: 0.32758293838862557


## Second modele avec DNN

In [9]:
#1700

model = tf.keras.Sequential([
    tf.keras.layers.Dense(850, activation='relu', input_shape=(max_features,)),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(35, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2023-11-22 17:19:42.473763: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 17:19:43.163630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 17:19:43.163744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train_trans, y_train, epochs=10, batch_size=32)

Epoch 1/10


2023-11-22 17:19:45.297017: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f38e58af4b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 17:19:45.297038: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-22 17:19:45.340343: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 17:19:45.414105: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1700669985.487801  756023 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1589/1589 [==============================] - 5s 2ms/step - loss: 0.3470 - accuracy: 0.8302
Epoch 2/10
1589/1589 [==============================] - 2s 2ms/step - loss: 0.3000 - accuracy: 0.8600
Epoch 3/10
1589/1589 [==============================] - 2s 2ms/step - loss: 0.2062 - accuracy: 0.9155
Epoch 4/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0869 - accuracy: 0.9697
Epoch 5/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0452 - accuracy: 0.9845
Epoch 6/10
1589/1589 [==============================] - 2s 2ms/step - loss: 0.0325 - accuracy: 0.9892
Epoch 7/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0256 - accuracy: 0.9916
Epoch 8/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0230 - accuracy: 0.9929
Epoch 9/10
1589/1589 [==============================] - 2s 2ms/step - loss: 0.0218 - accuracy: 0.9934
Epoch 10/10
1589/1589 [==============================] - 2s 2ms/step - loss: 0.0181 - accurac

In [11]:
y_pred = model.predict(X_valid_trans)
y_pred = np.where(y_pred>=0.5, 1, 0)

print(f1_score(y_valid, y_pred))

 66/681 [=>............................] - ETA: 0s 

681/681 [==============================] - 1s 724us/step
0.48330216403954046
